In [1]:
!pip install -U rightmove-webscraper

You should consider upgrading via the 'c:\users\pasho\appdata\local\programs\python\python38-32\python.exe -m pip install --upgrade pip' command.



  Attempting uninstall: numpy
    Found existing installation: numpy 1.20.3
    Uninstalling numpy-1.20.3:
      Successfully uninstalled numpy-1.20.3
  Attempting uninstall: requests
    Found existing installation: requests 2.25.1
    Uninstalling requests-2.25.1:
      Successfully uninstalled requests-2.25.1
  Attempting uninstall: pandas
    Found existing installation: pandas 1.2.4
    Uninstalling pandas-1.2.4:
      Successfully uninstalled pandas-1.2.4
  Attempting uninstall: lxml
    Found existing installation: lxml 4.6.3
    Uninstalling lxml-4.6.3:
      Successfully uninstalled lxml-4.6.3


In [1]:
import bs4
import urllib.request
import csv
import requests
# import folium
# from folium.plugins import MarkerCluster
# from geopy.geocoders import Nominatim
import random
import time

import pandas as pd
import numpy as np
from rightmove_webscraper import RightmoveData

In [2]:
# TODO
# House data
    # Get data from websites
    # check if property value correct or overpriced based on previously sold houses in the 3 adjecent streets in the last 6 months to 2 years
# Mortgage
    # Calculate maximum years of mortgage
    # Calculate the amount of loan that will be able to be taken and monthly repaiment/interest rate
    # Calculate maximum property value/property affordability based on mortgage and deposit

# https://github.com/toby-p/rightmove_webscraper.py
# https://blog.apify.com/is-web-scraping-legal/#terms-of-use-and-scraping
# http://rightmove.co.uk/robots.txt
# http://www.rightmove.co.uk/sitemap.xml

In [45]:
def url_builder(identifier='REGION'):
    _locations = locations[identifier]
    for loc in _locations:
        url = f'https://www.rightmove.co.uk/property-for-sale/find.html?searchType={sale_or_rent}&locationIdentifier={identifier}%{loc}&insId=1&radius={radius}&minPrice={min_price}&maxPrice={max_price}&minBedrooms={min_bed}&maxBedrooms={max_bed}&displayPropertyType=&maxDaysSinceAdded=14&_includeSSTC=on&sortByPriceDescending=&primaryDisplayPropertyType=&secondaryDisplayPropertyType=&oldDisplayPropertyType=&oldPrimaryDisplayPropertyType=&newHome=&auction=false'
        yield url

def dataset_builder():
    outputset = pd.DataFrame()
    url = url_builder()
    data = RightmoveData(url)._get_results()
    outputset = pd.concat([outputset, data])
    
def ifisdigit(feature):
    s_list = [int(s) for s in feature.split() if s.isdigit()]
    if s_list:
        return True
    return False

In [48]:
property_data = pd.DataFrame(columns=['property link', 'property address', 'property value',
    'offer type', 'property type', 'bedrooms', 'bathrooms', 'blueprint link', 'agency name', 
    'agency address', 'agency link', 'agency telephone', 'contact form', 'features', 
    'property description', 'cash', 'shared', 'share value', 'auction', 'ECP link'])

data = pd.DataFrame()

sale_or_rent = 'SALE' # 'RENT'
location = '5E2205'

# https://www.reddit.com/r/nottingham/comments/9mo4ou/nice_areas_to_live_around_nottingham/
# OUTCODE
NG1 = '5E1752' # NG1 - Centre 
NG2 = '5E1763' # NG2 - West bridgeford
NG3 = '5E1770' # NG3 - Saint ann, sneigton
NG4 = '5E1775' # NG4 - Netherfield
NG5 = '5E1776' # NG5 - Arnold
NG6 = '5E1777' # NG6 - Bulwell
NG7 = '5E1778' # NG7 - The park
NG8 = '5E1780' # NG8 - Hyson Green
NG9 = '5E1781' # NG9 - Beston
# REGION
Arnold = '5E2205'
Beeston = '5E3205'
Mapperley = '5E93530'
ThePark = '5E24206'
# Ruddington = '5E21243'
WestBridgford = '5E26169'
# Woodborough = '5E27299'
Carlton = '5E5612'
Wollaton = '5E27250'

locations = {'OUTCODE': [NG1, NG2, NG3,
                        NG4, NG5, NG6,
                        NG7, NG8, NG9],
             'REGION': [Arnold, Beeston, Mapperley,
                       ThePark, WestBridgford, Carlton,
                       Wollaton]}

# search properties
min_bed = 2 # bedrooms
max_bed = 4 # bedrooms
min_price = 0 # pounds
max_price = 200000 # pounds
radius = 1 # mile
# property_type = 'house'
# distance_list = [0.25, 0.5, 1, 3, 5, 10, 15, 30, 40, 50, 75, 100]

urls = url_builder()
for url in urls:
    temporary_data = RightmoveData(url)._get_results()
    data = pd.concat([data, temporary_data], ignore_index=True)

In [59]:
data.drop_duplicates(subset=['url'], inplace=True)

In [60]:
for link in data['url']:

    property_base_url = link[:link.find('#')]
    agencies_base_url = 'http://www.rightmove.co.uk/'

    r = requests.get(property_base_url)
    soup = bs4.BeautifulSoup(r.content)

    property_address = soup.find(class_="_2uQQ3SV0eMHL1P6t5ZDo2q").text if soup.find(class_="_2uQQ3SV0eMHL1P6t5ZDo2q").text else None # address
    property_value = soup.find(class_="_1gfnqJ3Vtd1z40MlC0MzXu").text if soup.find(class_="_1gfnqJ3Vtd1z40MlC0MzXu") else None
    offer_type = soup.find(class_="_16Ww3GEBGqbC4V9u-pm3MT").text if soup.find(class_="_16Ww3GEBGqbC4V9u-pm3MT") else None # price class
    property_type = soup.find_all(class_="_1hV1kqpVceE9m-QrX_hWDN")[0].text if len(soup.find_all(class_="_1hV1kqpVceE9m-QrX_hWDN")) > 0 else None # property type
    property_bedrooms = soup.find_all(class_="_1hV1kqpVceE9m-QrX_hWDN")[1].text[1:] if len(soup.find_all(class_="_1hV1kqpVceE9m-QrX_hWDN")) > 1 else None # bedrooms
    property_bathrooms = soup.find_all(class_="_1hV1kqpVceE9m-QrX_hWDN")[2].text[1:] if len(soup.find_all(class_="_1hV1kqpVceE9m-QrX_hWDN")) > 2 else None # bathrooms
    blueprint_url = property_base_url + soup.find(class_='_1EKvilxkEc0XS32Gwbn-iU').get('href') if soup.find(class_='_1EKvilxkEc0XS32Gwbn-iU') else None # blueprint link


    # agency details
    agency_name = soup.find(class_="_3PpywCmRYxC0B-ShNWxstv").text if soup.find(class_="_3PpywCmRYxC0B-ShNWxstv") else None # agency name
    agency_address = soup.find(class_="_1zJF3rohTQLpqNFDBD59qt").text if soup.find(class_="_1zJF3rohTQLpqNFDBD59qt") else None # agency address
    agency_link = agencies_base_url+soup.find(class_="YrIZhB0KLTi_J-sEv5dJa").get('href') if soup.find(class_="YrIZhB0KLTi_J-sEv5dJa") else None # agency link
    agency_telephone = soup.find(class_="_3fRacUNSjSxx6ntRK-jShL").get('href') if soup.find(class_="_3fRacUNSjSxx6ntRK-jShL") else None # telephone
    agency_contact_form = agencies_base_url + soup.find(class_="_3RZrxRVj_9Dn-Ot3w1ZjcX _1bauMPjK37TtcFKTHFxjlF").get('href') if soup.find(class_="_3RZrxRVj_9Dn-Ot3w1ZjcX _1bauMPjK37TtcFKTHFxjlF") else None # contact form

    key_features = [li.text for li in soup.find(class_='_1uI3IvdF5sIuBtRIvKrreQ').children] if soup.find(class_='_1uI3IvdF5sIuBtRIvKrreQ') else None #key features
    property_description = soup.find(class_='STw8udCxUaBUMfOOZu0iL').text if soup.find(class_='STw8udCxUaBUMfOOZu0iL') else None # text



    # cash buyers
    cash = None
    if key_features:
        possible_cash = [feature for feature in key_features if all(x in feature for x in ['cash', 'only'])]
        if possible_cash:
            cash = possible_cash
    else:
        if all(x in property_description for x in ['cash', 'only']):
            cash = True

    # shared ownership, percent, total property value
    share_value = None
    shared = True if 'shared ownership' in property_description.lower() else None # or 'shared ownership' in 
    if shared:
        value_index = property_description.lower().find('market value') if property_description.lower().find('market value') else ''
        s = property_description[value_index:value_index+50]
        s = s.replace(',', '').replace('£', '')
        property_value = [int(s) for s in s.split() if s.isdigit()][0] if [int(s) for s in s.split() if s.isdigit()] else None # total property value

        if key_features:
            possible_share = [feature for feature in key_features if 'share' in feature and ifisdigit(feature)]
            if possible_share:
                share_value = possible_share
        if not share_value:
            share_value_position = property_description.lower().find('% share') if property_description.lower().find('% share') else ''
            share_value = property_description[share_value_position-2:share_value_position+20] if share_value_position else None

    auction = True if 'auction' in property_description.lower() else 'N/A' # auction
    ECP_link = soup.find(class_="_3z91CUtoOLTHRPqcxFHtvn").get('href') if soup.find(class_="_3z91CUtoOLTHRPqcxFHtvn") else None # EPC rating

    temporary_property_data = pd.DataFrame(data={'property link': property_base_url, 'property address': property_address, 'property value': property_value,
        'offer type': offer_type, 'property type': property_type, 'bedrooms': property_bedrooms, 'bathrooms': property_bathrooms,
        'blueprint link': blueprint_url, 'agency name': agency_name, 'agency address': agency_address, 'agency link': agency_link,
        'agency telephone': agency_telephone, 'contact form': agency_contact_form, 'features': [key_features], 
        'property description': property_description, 'cash': cash, 'shared': shared, 'share value': share_value,
        'auction': auction, 'ECP link': ECP_link})
    
    property_data = pd.concat([property_data, temporary_property_data], ignore_index=True)

# freehold/Leasehold
# market info - last bought/sold
# council tax - from council website based on address
# check number of bedrooms in land registry
# check land registry for information
    
# soup.find(class_
# soup.find(class_


In [62]:
property_data.to_excel("sales.xlsx") 

--------------------------------------------------------------------------------------

--------------------------------------------------------------------------------------

--------------------------------------------------------------------------------------

--------------------------------------------------------------------------------------

# Find properties/area/street demographics

In [ ]:
https://www.streetcheck.co.uk/postcode/ng32bp
https://niceareas.co.uk/crime-map/for/nottingham/city/ng/ng1/ng1-5/crime-types/all/time-period/last-30-days/52.956200/-1.151200/
https://www.police.uk/pu/your-area/nottinghamshire-police/st-anns/?tab=CrimeMap

## For buy property

In [17]:
# Mortgage

# https://www.moneysavingexpert.com/mortgages/best-buys/?journeyType=first-time-buyer&propertyValue=115000&depositAmount=20000&term=30&introTerms=Fixed&repaymentMethod=Repayment&sortBy=MonthlyRepaymentAmount&pageNumber=1&addFeeToBalance=true&productNoFee=false&noEarlyRepaymentCharge=false

max_loan_age = 70

Total_annual_income_1 = 25000 # including salary and additional side income
Total_annual_income_2 = 0

Deposit = 20000
Budget_for_fees = 0
Budget_for_repairs = 0

Age_1 = 24
Age_2 = 53

Max_loan_duration = max_loan_age - max(Age_1, Age_2)

# TODO scrape website for correct range based on above details
Loan = (Total_annual_income_1+Total_annual_income_2)*4.5

Interest_rate = 2.5/100

Monthly_payment = Loan*Interest_rate

Maximum_house_price_availability = Loan+Deposit

In [15]:
Maximum_house_price_availability

132500.0

In [ ]:
# Calculates the annual yield of a rented property

Annual_rental_income = 8400
Prorperty_Purchase_price= 150.000
Property_current_price = []

annual_yield = 100*Annual_rental_income/Purchase_price

if annual_yield < 7:
    print(f'Current annual yield of {annual_yield} is below the ideal return!')
else:
    print(f'The current annual yield of {annual_yield} is performining extremely well!')

In [475]:
# https://github.com/toby-p/rightmove_webscraper.py/blob/cdb79a237d3e21ee7376dc88ad717af9b39b27e3/rightmove_webscraper/scraper.py#L151

import bs4
import requests

status_code, content = requests.get(url)
soup = bs4.BeautifulSoup(content)